In [1]:
"""
- data_folder
--- eye_tracking_folder
--- aoi_folder

data에는 timestamp, x, y 칼럼이 존재함.
"""

'\n- data_folder\n--- eye_tracking_folder\n--- aoi_folder\n\ndata에는 timestamp, x, y 칼럼이 존재함.\n'

In [2]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
import seaborn as sns

In [3]:
"""
coordinate_data = {
        'Timestamp': ts,
        'X':x,
        'Y': y
    }

"""

"\ncoordinate_data = {\n        'Timestamp': ts,\n        'X':x,\n        'Y': y\n    }\n\n"

In [4]:
def pass_sav_gol_filter(coordinate_data):
    
    coordinate_data['X'] = savgol_filter(coordinate_data['X'], 5, 3)
    coordinate_data['Y'] = savgol_filter(coordinate_data['Y'], 5, 3)

    return coordinate_data
    
def get_angular_velocity(coordinate_data, px2deg):
    x = coordinate_data['X']
    y = coordinate_data['Y']
    ts = coordinate_data['Timestamp']
    velocity = np.array([np.NaN])

    for i in range(len(x)-1):
        degree = px2deg *  math.sqrt(((x[i+1] - x[i]) ** 2) + ((y[i+1] - y[i]) ** 2))

        velocity = np.append(
            velocity,
            (degree / (ts[i+1] - ts[i]) * 1000)
        )

    return velocity

def classify_events(velocity_data, velocity_threshold = 30):

    velocity_threshold = velocity_threshold
    # For adaptive velocity_threshold
    # velocity_threshold = find_adaptive_threshold(velocity_data, velocity_threshold)

    saccades = np.where(velocity_data > velocity_threshold)[0]
    fixations = np.where(velocity_data <= velocity_threshold)[0]
    blinks = np.where(np.isnan(velocity_data))[0][1:]

    events = {'saccades': saccades,
              'fixations': fixations,
              'blinks': blinks}

    for event_key, event in events.items():

        start_index = -1
        end_index = -1
        temp_result = []
        result = []
        for i in range(len(event)):
            if i != (len(event) - 1):
                if ((event[i] + 1) == event[i+1]) and (start_index == -1):
                    start_index = i
                elif ((event[i] + 1) == event[i+1]) and (start_index != -1):
                    continue
                elif ((event[i] + 1) != event[i+1]) and (start_index != -1):
                    end_index = i+1
                    temp_result = event[start_index: end_index]
                    result.append(temp_result)
                    start_index = -1
                    end_index = -1
                    temp_result = []
                elif ((event[i] + 1) != event[i+1]) and (start_index == -1):
                    result.append([event[i]])
            else:
                if start_index == -1:
                    result.append([event[i]])
                elif (start_index != -1) and ((event[i] - 1) == event[i - 1]):
                    end_index = i
                    result.append(event[start_index:])

        events[event_key] = result

    return events

# For adaptive velocity threshold
def update_threshold(velocity_data, threshold):
    velocity_below_threshold = velocity_data[np.where(velocity_data < threshold)]
    mean = velocity_below_threshold.mean()
    std = velocity_below_threshold.std()
    updated_threshold = mean + (std * 6)

    return updated_threshold

def find_adaptive_threshold(velocity_data, threshold):
    while abs(update_threshold(velocity_data, threshold) - threshold) > 1:
        threshold = update_threshold(velocity_data, threshold)
    return threshold

def get_classification_result(coordinate_data, velocity_data):
    result = classify_events(velocity_data)
    timestamp, x, y = coordinate_data['Timestamp'], coordinate_data['X'], coordinate_data['Y']
    output = []
    for key, value in result.items():
        for i in value:
            start_time = timestamp[i[0]-1]
            end_time = timestamp[i[-1]]
            start_x = x[i[0]-1]
            end_x = x[i[-1]]
            start_y = y[i[0]-1]
            end_y = y[i[-1]]
            output.append([key, start_time, end_time, (end_time-start_time), start_x, start_y, end_x, end_y])
    df_output = pd.DataFrame(data = output,
                             columns = ['label', 'start_time', 'end_time', 'duration', 'start_x', 'start_y', 'end_x', 'end_y'])
    df_output.sort_values(by=['start_time'], axis=0, inplace=True)
    return df_output


    
    
    

class KirbsLai:
    def __init__(self, test_name, screen_size, distance, screen_resolution):
        self.test_name = test_name
        self.current_dir = os.getcwd() + '\\{}'.format(test_name)
        self.input_dir = self.current_dir+ '\\input'
        self.aoi_dir = self.current_dir+ '\\aoi'
        self.faces_aoi = pd.read_csv(self.aoi_dir+'\\faces.csv', index_col = 0)
        self.identification_aoi = pd.read_csv(self.aoi_dir+'\\identification.csv', index_col = 0)
        self.px2deg = math.degrees(math.atan2(0.5 * screen_size, distance)) / (0.5 * screen_resolution)
        self.input_list = os.listdir(self.input_dir)
        self.module_list = ['identification', 'faces']
        
    def load_input(self, input_file_name):
        temp_data = pd.read_csv(self.input_dir + '\\{}'.format(input_file_name))
        
        return temp_data
    
    def get_identification_fixation(self, temp_result, item_num):
        eye_x_aoi_1 = (temp_result["start_x"] >= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'x1'])
        eye_x_aoi_2 = (temp_result["start_x"] <= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'x2'])

        eye_x_aoi = (eye_x_aoi_1 & eye_x_aoi_2)

        eye_y_aoi_1 = (temp_result["start_y"] >= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'y1'])
        eye_y_aoi_2 = (temp_result["start_y"] <= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'y2'])

        eye_y_aoi = (eye_y_aoi_1 & eye_y_aoi_2)

        mouth_x_aoi_1 = (temp_result["start_x"] >= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'x1'])
        mouth_x_aoi_2 = (temp_result["start_x"] <= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'x2'])

        mouth_x_aoi = (mouth_x_aoi_1 & mouth_x_aoi_2)

        mouth_y_aoi_1 = (temp_result["start_y"] >= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'y1'])
        mouth_y_aoi_2 = (temp_result["start_y"] <= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'y2'])

        mouth_y_aoi = (mouth_y_aoi_1 & mouth_y_aoi_2)

        fixation_in_eye_aoi = temp_result[eye_x_aoi & eye_y_aoi]
        fixation_in_mouth_aoi = temp_result[mouth_x_aoi & mouth_y_aoi]

        return fixation_in_eye_aoi, fixation_in_mouth_aoi

    def get_faces_fixation(self, temp_result, item_num):
        face11_x_aoi = ((temp_result["start_x"] > self.faces_aoi.at['faces_11', 'x1']) & (temp_result["start_x"] < self.faces_aoi.at['faces_11', 'x2']))
        face11_y_aoi = ((temp_result["start_y"] > self.faces_aoi.at['faces_11', 'y1']) & (temp_result["start_y"] < self.faces_aoi.at['faces_11', 'y2']))

        face12_x_aoi = ((temp_result["start_x"] > self.faces_aoi.at['faces_12', 'x1']) & (temp_result["start_x"] < self.faces_aoi.at['faces_12', 'x2']))
        face12_y_aoi = ((temp_result["start_y"] > self.faces_aoi.at['faces_12', 'y1']) & (temp_result["start_y"] < self.faces_aoi.at['faces_12', 'y2']))

        face21_x_aoi = ((temp_result["start_x"] > self.faces_aoi.at['faces_21', 'x1']) & (temp_result["start_x"] < self.faces_aoi.at['faces_21', 'x2']))
        face21_y_aoi = ((temp_result["start_y"] > self.faces_aoi.at['faces_21', 'y1']) & (temp_result["start_y"] < self.faces_aoi.at['faces_21', 'y2']))

        face22_x_aoi = ((temp_result["start_x"] > self.faces_aoi.at['faces_22', 'x1']) & (temp_result["start_x"] < self.faces_aoi.at['faces_22', 'x2']))
        face22_y_aoi = ((temp_result["start_y"] > self.faces_aoi.at['faces_22', 'y1']) & (temp_result["start_y"] < self.faces_aoi.at['faces_22', 'y2']))

        total_fixation = temp_result.label.count()
        face11_fixation = temp_result[face11_x_aoi & face11_y_aoi].label.count()
        face12_fixation = temp_result[face12_x_aoi & face12_y_aoi].label.count()
        face21_fixation = temp_result[face21_x_aoi & face21_y_aoi].label.count()
        face22_fixation = temp_result[face22_x_aoi & face22_y_aoi].label.count()

        return total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation

    def extract_identification_dwelltime(self, df_temp, item_num):
        trial_duration = max(df_temp['timestamp'])
        all_point_in_trial = df_temp.shape[0]    

        eye_x_aoi_1 = (df_temp["x"] >= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'x1'])
        eye_x_aoi_2 = (df_temp["x"] <= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'x2'])

        eye_x_aoi = eye_x_aoi_1 & eye_x_aoi_2

        eye_y_aoi_1 = (df_temp["y"] >= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'y1'])
        eye_y_aoi_2 = (df_temp["y"] <= self.identification_aoi.at['item_num_{}_eye'.format(item_num), 'y2'])

        eye_y_aoi = eye_y_aoi_1 & eye_y_aoi_2

        mouth_x_aoi_1 = (df_temp["x"] >= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'x1'])
        mouth_x_aoi_2 = (df_temp["x"] <= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'x2'])

        mouth_x_aoi = mouth_x_aoi_1 & mouth_x_aoi_2

        mouth_y_aoi_1 = (df_temp["y"] >= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'y1'])
        mouth_y_aoi_2 = (df_temp["y"] <= self.identification_aoi.at['item_num_{}_mouth'.format(item_num), 'y2'])

        mouth_y_aoi = mouth_y_aoi_1 & mouth_y_aoi_2

        point_in_eye_aoi = df_temp[eye_x_aoi & eye_y_aoi]
        point_in_mouth_aoi = df_temp[mouth_x_aoi & mouth_y_aoi]

        abs_dwell_time_eyes = len(point_in_eye_aoi) / all_point_in_trial * trial_duration
        abs_dwell_time_mouth = len(point_in_mouth_aoi) / all_point_in_trial * trial_duration

        try:
            first_time_eye = df_temp[eye_x_aoi & eye_y_aoi].iat[0,4]
        except:
            first_time_eye = np.nan

        return abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye
    
    def extract_faces_dwelltime(self, df_temp, item_num):

        trial_duration = max(df_temp['timestamp'])
        all_point_in_trial = df_temp.shape[0]   

        face11_x_aoi = (df_temp["x"] > self.faces_aoi.at['faces_11', 'x1']) & (df_temp["x"] < self.faces_aoi.at['faces_11', 'x2'])
        face11_y_aoi = (df_temp["y"] > self.faces_aoi.at['faces_11', 'y1']) & (df_temp["y"] < self.faces_aoi.at['faces_11', 'y2'])

        face12_x_aoi = (df_temp["x"] > self.faces_aoi.at['faces_12', 'x1']) & (df_temp["x"] < self.faces_aoi.at['faces_12', 'x2'])
        face12_y_aoi = (df_temp["y"] > self.faces_aoi.at['faces_12', 'y1']) & (df_temp["y"] < self.faces_aoi.at['faces_12', 'y2'])

        face21_x_aoi = (df_temp["x"] > self.faces_aoi.at['faces_21', 'x1']) & (df_temp["x"] < self.faces_aoi.at['faces_21', 'x2'])
        face21_y_aoi = (df_temp["y"] > self.faces_aoi.at['faces_21', 'y1']) & (df_temp["y"] < self.faces_aoi.at['faces_21', 'y2'])

        face22_x_aoi = (df_temp["x"] > self.faces_aoi.at['faces_22', 'x1']) & (df_temp["x"] < self.faces_aoi.at['faces_22', 'x2'])
        face22_y_aoi = (df_temp["y"] > self.faces_aoi.at['faces_22', 'y1']) & (df_temp["y"] < self.faces_aoi.at['faces_22', 'y2'])

        point_in_face11_aoi = df_temp[face11_x_aoi & face11_y_aoi]
        point_in_face12_aoi = df_temp[face12_x_aoi & face12_y_aoi]
        point_in_face21_aoi = df_temp[face21_x_aoi & face21_y_aoi]
        point_in_face22_aoi = df_temp[face22_x_aoi & face22_y_aoi]
        
        abs_dwell_time_face11 = len(point_in_face11_aoi) / all_point_in_trial * trial_duration
        abs_dwell_time_face12 = len(point_in_face12_aoi) / all_point_in_trial * trial_duration
        abs_dwell_time_face21 = len(point_in_face21_aoi) / all_point_in_trial * trial_duration
        abs_dwell_time_face22 = len(point_in_face22_aoi) / all_point_in_trial * trial_duration
        
        return abs_dwell_time_face11, abs_dwell_time_face12, abs_dwell_time_face21, abs_dwell_time_face22
    
    
    def get_result(self, input_file_name):
        identification_result = pd.DataFrame(columns = [
            'participant',            
            'module',
            'trial',
            'total_fixation',
            'eye_fixation',
            'mouth_fixation',
            'abs_dwell_time_eyes',
            'abs_dwell_time_mouth',
            'first_time_eye'])

        faces_result = pd.DataFrame(columns = [
            'participant',
            'module',
            'trial',
            'total_fixation',
            'face11_fixation',
            'face12_fixation',
            'face21_fixation',
            'face22_fixation',
            'total_fixation_1000',
            'face11_fixation_1000',
            'face12_fixation_1000',
            'face21_fixation_1000',
            'face22_fixation_1000',
            'abs_dwell_time_face11',
            'abs_dwell_time_face12',
            'abs_dwell_time_face21',
            'abs_dwell_time_face22'])
        
        participant = input_file_name[-12:-4]
        identification_aoi = self.identification_aoi
        faces_aoi = self.faces_aoi
        data = self.load_input(input_file_name)
        for module in self.module_list:
            for trial in [0, 2, 4, 6, 8]:
                temp_data = data[(data.trial == trial) &\
                             (data.module == module)]
                temp_data = temp_data.drop_duplicates(['timestamp', 'module', 'trial'], keep='last').reset_index().copy()
                
                
                ts, x, y = np.array(temp_data['timestamp']), np.array(temp_data['x']), np.array(temp_data['y'])
                temp_data_dict = {
                    'Timestamp': ts,
                    'X':x,
                    'Y': y
                }

                temp_data_dict = pass_sav_gol_filter(temp_data_dict)
                velocity = get_angular_velocity(temp_data_dict, self.px2deg)
                temp_result = get_classification_result(temp_data_dict, velocity)
                temp_result = temp_result[temp_result.label == 'fixations']
                
                
                
                # 아래 코드는 1초 미만일 때의 Fixation을 찾기 위해 사용할 temp_result_1000을 만들기 위한 코드입니다.
                # temp_result 코드와 모두 동일하나 1초 미만의 데이터만 포함시킨다는 차이가 있습니다.
                
                temp_data_1000 = data[(data.trial == trial) &\
                             (data.module == module)]
                temp_data_1000 = temp_data_1000.drop_duplicates(['timestamp', 'module', 'trial'], keep='last').reset_index().copy()
                
                temp_data_1000 = temp_data_1000[temp_data_1000.timestamp <= 1000]
                
                ts, x, y = np.array(temp_data_1000['timestamp']), np.array(temp_data_1000['x']), np.array(temp_data_1000['y'])
                temp_data_1000_dict = {
                    'Timestamp': ts,
                    'X':x,
                    'Y': y
                }
                
                temp_data_1000_dict = pass_sav_gol_filter(temp_data_1000_dict)
                velocity = get_angular_velocity(temp_data_1000_dict, self.px2deg)
                temp_result_1000 = get_classification_result(temp_data_1000_dict, velocity)
                temp_result_1000 = temp_result_1000[temp_result_1000.label == 'fixations']
            
                
                # 나중에 trial과 item_num이 다를 경우에는 아래 코드 수정해야 함.
                # trial과 item_num을 짝지어주는 코드 입력하면 됨.
                item_num = trial // 2
        
                if module == 'identification':

                    fixation_in_eye_aoi, fixation_in_mouth_aoi = self.get_identification_fixation(temp_result, item_num)
                    abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye = self.extract_identification_dwelltime(temp_data, item_num)

                    total_fixation = temp_result.label.count()
                    eye_fixation = fixation_in_eye_aoi.label.count()
                    mouth_fixation = fixation_in_mouth_aoi.label.count()

                    temp_df = pd.DataFrame({
                        'participant': participant,
                        'module': module,
                        'trial': trial,
                        'total_fixation': total_fixation,
                        'eye_fixation': eye_fixation,
                        'mouth_fixation': mouth_fixation,
                        'abs_dwell_time_eyes': abs_dwell_time_eyes,
                        'abs_dwell_time_mouth': abs_dwell_time_mouth,
                        'first_time_eye': first_time_eye}, index=[0])
                    
                    identification_result = pd.concat([identification_result, temp_df])

                elif module == 'faces':
                    """
                    자극 시간 전체에 걸쳐 AOI 내에 들어온 Fixation을 찾는 코드입니다.
                    """
                    total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation =\
                    self.get_faces_fixation(temp_result_1000, item_num)
                    
                    """
                    첫 1초 동안 AOI 내에 들어온 Fixation을 찾는 코드입니다.
                    """
                    total_fixation_1000, face11_fixation_1000, face12_fixation_1000, face21_fixation_1000, face22_fixation_1000 =\
                    self.get_faces_fixation(temp_result_1000, item_num)
                    
                    abs_dwell_time_face11, abs_dwell_time_face12, abs_dwell_time_face21, abs_dwell_time_face22 = self.extract_faces_dwelltime(temp_data, item_num)

                    temp_df = pd.DataFrame({
                                            'participant': participant,
                                            'module': module,
                                            'trial': trial,
                                            'total_fixation': total_fixation,
                                            'face11_fixation': face11_fixation,
                                            'face12_fixation': face12_fixation,
                                            'face21_fixation': face21_fixation,
                                            'face22_fixation': face22_fixation,
                                            'total_fixation_1000': total_fixation_1000,
                                            'face11_fixation_1000': face11_fixation_1000,
                                            'face12_fixation_1000': face12_fixation_1000,
                                            'face21_fixation_1000': face21_fixation_1000,
                                            'face22_fixation_1000': face22_fixation_1000,
                                            'abs_dwell_time_face11': abs_dwell_time_face11,
                                            'abs_dwell_time_face12': abs_dwell_time_face12,
                                            'abs_dwell_time_face21': abs_dwell_time_face21,
                                            'abs_dwell_time_face22': abs_dwell_time_face22}, index = [0])
                    faces_result = pd.concat([faces_result, temp_df])
                    
        return identification_result, faces_result

In [7]:
test = KirbsLai('kirbs_20220518', 62, 60, 1920)

In [8]:
test.px2deg

0.02846238677223598

In [9]:
result1, result2 = test.get_result(test.input_list[3])

In [14]:
identification_result = pd.DataFrame(columns = [
                                        'participant',            
                                        'module',
                                        'trial',
                                        'total_fixation',
                                        'eye_fixation',
                                        'mouth_fixation'])

faces_result = pd.DataFrame(columns = [
                                        'participant',
                                        'module',
                                        'trial',
                                        'total_fixation',
                                        'face11_fixation',
                                        'face12_fixation',
                                        'face21_fixation',
                                        'face22_fixation',
                                        'total_fixation_1000',
                                        'face11_fixation_1000',
                                        'face12_fixation_1000',
                                        'face21_fixation_1000',
                                        'face22_fixation_1000'])
for i in test.input_list:
    temp_identification, temp_faces = test.get_result(i)
    
    identification_result = pd.concat([identification_result, temp_identification])
    faces_result = pd.concat([faces_result, temp_faces])

identification_result.reset_index(drop = True, inplace = True)
faces_result.reset_index(drop = True, inplace = True)

In [15]:
identification_result

,participant,module,trial,total_fixation,eye_fixation,mouth_fixation,abs_dwell_time_eyes,abs_dwell_time_mouth,first_time_eye
0,00000000,identification,0,1,0,0,2902.292308,0.000000,153
1,00000000,identification,2,4,0,0,728.906250,473.789062,495
2,00000000,identification,4,5,0,1,642.229008,285.435115,2105
3,00000000,identification,6,3,0,1,1402.139706,1880.919118,3248
4,00000000,identification,8,4,0,1,3566.281250,109.171875,521
5,00000001,identification,0,2,0,0,548.267717,1352.393701,554
6,00000001,identification,2,4,0,1,259.448000,963.664000,2750
7,00000001,identification,4,5,1,0,526.422764,1729.674797,321
8,00000001,identification,6,1,0,0,0.000000,330.857143,NaN
9,00000001,identification,8,1,0,0,148.448000,2375.168000,636


In [19]:
faces_result

,participant,module,trial,total_fixation,face11_fixation,face12_fixation,face21_fixation,face22_fixation,total_fixation_1000,face11_fixation_1000,face12_fixation_1000,face21_fixation_1000,face22_fixation_1000,abs_dwell_time_face11,abs_dwell_time_face12,abs_dwell_time_face21,abs_dwell_time_face22
0,00000000,faces,0,2,1,0,0,0,2,1,0,0,0,1159.000000,729.740741,643.888889,42.925926
0,00000000,faces,2,3,0,1,1,0,3,0,1,1,0,1031.777778,884.380952,1068.626984,36.849206
0,00000000,faces,4,3,1,0,1,0,3,1,0,1,0,1353.551724,477.724138,1035.068966,0.000000
0,00000000,faces,6,2,1,0,0,0,2,1,0,0,0,1486.607143,412.946429,991.071429,1156.250000
0,00000000,faces,8,3,2,0,1,0,3,2,0,1,0,809.825243,809.825243,989.786408,0.000000
0,00000001,faces,0,5,1,1,0,1,5,1,1,0,1,1205.273438,1205.273438,438.281250,730.468750
0,00000001,faces,2,5,1,1,0,2,5,1,1,0,2,801.519231,1469.451923,712.461538,1291.336538
0,00000001,faces,4,3,1,1,0,0,3,1,1,0,0,1469.733871,1582.790323,565.282258,602.967742
0,00000001,faces,6,4,2,0,1,0,4,2,0,1,0,2277.767442,1193.116279,325.395349,578.480620
0,00000001,faces,8,3,1,1,0,0,3,1,1,0,0,2225.546875,1094.531250,218.906250,729.687500


In [90]:
identification_result

,participant,module,trial,total_fixation,eye_fixation,mouth_fixation,abs_dwell_time_eyes,abs_dwell_time_mouth,first_time_eye
0,00000000,identification,0,1,0,0,2902.292308,0.000000,0
0,00000000,identification,2,4,0,0,728.906250,473.789062,2
0,00000000,identification,4,5,0,1,642.229008,285.435115,4
0,00000000,identification,6,3,0,1,1402.139706,1880.919118,6
0,00000000,identification,8,4,0,1,3566.281250,109.171875,8
0,00000001,identification,0,2,0,0,548.267717,1352.393701,0
0,00000001,identification,2,4,0,1,259.448000,963.664000,2
0,00000001,identification,4,5,1,0,526.422764,1729.674797,4
0,00000001,identification,6,1,0,0,0.000000,330.857143,NaN
0,00000001,identification,8,1,0,0,148.448000,2375.168000,8
